methodology [here](https://hackmd.io/twOh6VgqSnyaAy8BbYrVLg?edit)

## installation and set up

In [21]:
!pip install openai

from IPython.display import clear_output

def clear():
  clear_output()


from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### text-to-text

In [22]:
from openai import OpenAI
from google.colab import userdata
import textwrap

def text_to_text(prompt):
    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    system_prompt = "You are a helpful assistant with a wealth of knowledge about folklore and myth from a variety of cultures across the globe."
    user_prompt = prompt
    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    wrapped_text = textwrap.fill(response.choices[0].message.content, width=500)
    print(wrapped_text)
    return response.choices[0].message.content

In [34]:
from openai import OpenAI
from google.colab import userdata
import textwrap

def safe_prompt(prompt):
    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    system_prompt = "You are a helpful assistant that modifies fairytales by sensoring grotesk descriptions"
    user_prompt = f"Please modify the following prompt {prompt} by sensoring grotesk descriptions. Do not take away from the essence of the story."
    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    wrapped_text = textwrap.fill(response.choices[0].message.content, width=500)
    #print(wrapped_text)
    return response.choices[0].message.content


### text-to-image

In [23]:
from openai import OpenAI
from google.colab import userdata
from IPython.display import Image, display
import requests
from datetime import datetime
import os

def text_to_image(prompt, output_folder="/content/drive/MyDrive/ll_color_colabs/_projects/_cl200_color_colabs/group_1/output"):
    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url
    print(f"Image URL: {image_url}")
    display(Image(url=image_url, width=500))
    if output_folder:
        # Fetch the image content
        image_content = requests.get(image_url).content

        # Generate a timestamp
        now = datetime.now()  # current date and time
        timestamp = now.strftime("%Y%m%d_%H%M%S")

        # Ensuring the output directory exists
        if not os.path.exists(output_folder):
            os.makedirs(output_folder, exist_ok=True)

        image_file_path = os.path.join(output_folder, f'eclipse_image_{timestamp}.png')

        # Save the image
        with open(image_file_path, 'wb') as image_file:
            image_file.write(image_content)

        print(f"Image saved to {image_file_path}")

    return image_url


### text-to-voice



In [24]:
from openai import OpenAI
from google.colab import userdata
from IPython.display import Audio, display
import os

def text_to_speech(input_text, voice_effect="shimmer", output_folder="/content/drive/MyDrive/ll_color_colabs/_projects/_cl200_color_colabs/group_1/output"):

    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

    # Generate a filename based on current datetime to avoid overwrite
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    speech_file_path = os.path.join(output_folder, f"speech_{voice_effect}_{timestamp}.mp3")

    # Ensuring the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)

    # Generate the audio for the input text
    response = client.audio.speech.create(
        model="tts-1",
        voice=voice_effect,
        input=input_text
    )

    # Save the audio to the specified path
    response.stream_to_file(speech_file_path)
    print(f"Audio saved to {speech_file_path}")

    # Play the audio in the notebook
    display(Audio(speech_file_path, autoplay=True))

# Example usage:


Now, we create a function that converts an image to text.

In [25]:
from google.colab import userdata
from openai import OpenAI
import base64
import os

def image_to_text(prompt, image_path_or_url, is_url=True):
    """
    Converts an image to text using OpenAI's API.

    Parameters:
    - prompt: A text prompt to guide the image description.
    - image_path_or_url: The path to the image file or an image URL.
    - is_url: A boolean indicating whether the image is provided as a URL or a local file path.
    """

    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

    if is_url:
        print("URL option selected")
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {"url": image_path_or_url},
                        },
                    ],
                }
            ],
            max_tokens=300,
        )
    else:
        print("Image File option selected")

        # Function to encode the image to base64
        def encode_image(image_path):
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')

        base64_image = encode_image(image_path_or_url)

        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                        },
                    ],
                }
            ],
            max_tokens=300,
        )

    print(response.choices[0].message.content)


In [38]:
# Create a story based on the prompt in passed into the text_to_text function
story = text_to_text("describe an epic eclipse myth for me")

# Revise the story that is passed to the image generator to avoid violating the DALLE's content safety policy
revised_story = safe_prompt(story)

# Create an image based on the story that is generated
image = text_to_image(revised_story)

# Create a speech file that narrates the generated story
text_to_speech(story)

# Read in the created image and describe it via text
final_text = image_to_text("describe this image", image, is_url=True)

One compelling epic eclipse myth hails from the rich tapestry of Hindu mythology, centered around the celestial event of the churning of the Milky Ocean, also known as Samudra Manthan. This myth is not only a fascinating story that captures the cosmic scale of Hindu mythological narratives, but it also embeds the reason behind the occurrence of solar and lunar eclipses within its vast, allegorical confines.  The story begins with the Devas (gods) and the Asuras (demons) deciding to collaborate
in churning the Milky Ocean to obtain Amrita, the nectar of immortality. This monumental task required the use of Mount Mandara as the churning rod and the serpent Vasuki as the churning rope. The gods and demons churned the ocean back and forth, thus releasing a number of treasures and finally, Dhanvantari, the divine physician, emerged carrying a pot full of Amrita.  However, a dispute arose over who would drink the nectar first. Lord Vishnu, seeing the potential chaos, took the form of an
ench

Image saved to /content/drive/MyDrive/ll_color_colabs/_projects/_cl200_color_colabs/group_1/output/eclipse_image_20240410_204920.png
Audio saved to /content/drive/MyDrive/ll_color_colabs/_projects/_cl200_color_colabs/group_1/output/speech_shimmer_20240410_204921.mp3


<ipython-input-24-617ca8b8383a>:27: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


URL option selected
This is a colorful and intricate representation of a scene from Hindu mythology known as the churning of the ocean or "Samudra Manthan." At the center of the image, there is a multi-armed figure, which is a depiction of Lord Vishnu in one of his avatars, possibly as Lord Dhanvantari or as a form related to the churning event. This central figure is surrounded by other deities and beings who are participating in the churning. 

On either side of the milky ocean, there are groups of figures pulling on the giant snake, which is being used as a churning rope. To the left are the Devas (gods), and to the right are the Asuras (demons).

The ocean is depicted as turbulent with waves, and it contains various mythological creatures and items emerging from the churning, including the goddess of prosperity (Lakshmi) and the celestial elephant (Airavata). The scene is flanked by mountains and the sky, which includes celestial beings and other mythical figures participating in o